In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import gpflow

import matplotlib
matplotlib.rcParams['figure.figsize'] = (14, 6)

# Classification with sparse GPs

### James Hensman 2017 - 2019

This notebook serves as a tutorial: the aim is to understand how GP classification with sparse approximations work in GPflow.

First, let's have a look at the illustrative dataset from the lecture. Here's how it was generated:

In [ ]:
# Draw a sample from a GP

# first build the kernel and kernel matrix
k = gpflow.kernels.Matern52(1, variance=6.0)
X_all = np.linspace(0, 6, 200).reshape(-1, 1)
K = k.compute_K_symm(X_all)

# sample from a multivariate normal
L = np.linalg.cholesky(K)
f_all = np.dot(L, np.random.RandomState(6).randn(200, 1))
plt.plot(X_all, f_all, 'C0')

# squash
p_all = np.exp(f_all) / (1 + np.exp(f_all))
plt.plot(X_all, p_all, 'C3')

# evaluate a small number of points
ind = np.random.randint(0, 200, (50,))
X = X_all[ind]
p = p_all[ind]
plt.plot(X, p, 'C3o', ms=6)


# bernoulli draws
Y = np.where(np.random.rand(50, 1) < p, 1, 0)
plt.plot(X, Y, 'C0x', ms=8, mew=2)

plt.xlim(0, 6)
plt.ylim(-1.5, 2.5)

### Exercise 1: the effect of the parameters on classification datasets

a) Change the variance parameter in the kernel above (try 100, 0.01). What is the effect on the data X, Y ?

b) What is the effect of the lengthscale parameter on X and Y?

Now let's build a GPflow model of these data.

In [ ]:
# here's a GPflow model. it assumes that the inducing locations Z are fixed to the data X.
# We'll tell the model that the data are Binary (Bernoulli likelihood), and we'll pick a kernel.
m = gpflow.models.SVGP(X, Y,
                      likelihood=gpflow.likelihoods.Bernoulli(),
                      kern=gpflow.kernels.Matern52(1),
                      Z=X.copy())
m.feature.Z.set_trainable(False)

o = gpflow.train.ScipyOptimizer()
o.minimize(m)

In [ ]:
def plot_1d(m):
    # work out some sensible limits
    xmin, xmax = m.X.read_value().min(), m.X.read_value().max()
    xmin, xmax = xmin - 0.1 * (xmax - xmin), xmax + 0.1 * (xmax - xmin)
    Xtest = np.linspace(xmin, xmax, 200).reshape(-1, 1)
    
    # bubble fill the predictions
    mu, var = m.predict_f(Xtest)
    plt.fill_between(Xtest.flatten(),
                     (mu + 2 * np.sqrt(var)).flatten(),
                     (mu - 2 * np.sqrt(var)).flatten(),
                     alpha=0.3, color='C1')
    
    # plot samples
    samples = m.predict_f_samples(Xtest, 10).squeeze()
    plt.plot(Xtest, samples.T, 'C1', lw=1)
    
    # plot p-samples
    p = np.exp(samples) / (1. + np.exp(samples))
    plt.plot(Xtest, p.T, 'C3', lw=1)

    # plot data
    plt.plot(m.X.read_value(), m.Y.read_value(), 'C0x', ms=8, mew=2)
    
    plt.xlim(xmin, xmax)

plot_1d(m)

Okay, now we have a model of your dataset. Let's poke the model and see if we can understand how it works. 

### Exercise 2: poke the model

a) Print the model. Can you relate the parameters to the variables discussed in the lecture?

b) After optimizing the model, has the model managed to estimate the kernel parameters effectively? 

c) Since we know what the optimal kernel parameters are, let's see how the model works with those. Assign the known kernel parameters to the model and mark them as not for training (`m.kern.foo.set_trainable(False)`). You will have to optimize the model again. Are these parameters better than the estimated ones? 

In [ ]:
# your answers here.

## Classification in 2D

It's straightforward to move the GP classification problem into 2D by simply changing the kernel. We'll do that here, and in addition relax the assumption that X=Z. 

In [ ]:
# here's a standard 2D dataset.
X_banana = np.loadtxt('data/banana_X_train', delimiter=',')
Y_banana = np.loadtxt('data/banana_Y_train', delimiter=',').reshape(-1, 1)

In [ ]:
# here's the sparse GP model:
m = gpflow.models.SVGP(X_banana, Y_banana,
                       kern=gpflow.kernels.Matern52(2),
                       likelihood=gpflow.likelihoods.Bernoulli(),
                       Z=np.random.rand(8, 2))

In [ ]:
# and here's a function for plotting the model
def plot_2d(m):
    sess = m.enquire_session(None)
    #plot the inducing point locations:
    if hasattr(m, 'Z'):
        Z = m.Z.read_value(sess)
        plt.plot(Z[:, 0], Z[:, 1], 'C3o', ms=10, label="inducing")
    
    xmin, ymin = m.X.read_value(sess).min(0)
    xmax, ymax = m.X.read_value(sess).max(0)
    xmin, xmax = xmin - 0.1 * (xmax - xmin), xmax + 0.1 * (xmax - xmin)
    ymin, ymax = ymin - 0.1 * (ymax - ymin), ymax + 0.1 * (ymax - ymin)
    
    xx, yy = np.mgrid[xmin:xmax:200j, ymin:ymax:200j]
    Xtest = np.vstack([xx.flatten(), yy.flatten()]).T
    mu, var = m.predict_y(Xtest)
    
    X, Y = m.X.read_value(sess), m.Y.read_value(sess)
    
    for i, level in [[0, 0.2], [1, 0.8]]:
        plt.plot(X[Y.flatten()==i, 0], X[Y.flatten()==i, 1], 'C{}o'.format(i), ms=8, label='y={}'.format(i))
        cs = plt.contour(xx, yy, mu.reshape(*xx.shape), [level], colors='C{}'.format(i), linewidths=3)
        cs.collections[0].set_label('p(y={}) = {}'.format(i, level))
                         
    cs = plt.contour(xx, yy, mu.reshape(*xx.shape), [0.5], colors='C3', linewidths=1)
    cs.collections[0].set_label('p(y=1) = 0.5')
    
    plt.legend(loc=0)
    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)


### Exercise 3: understanding the model

a) Use the above function to plot the model. What's going on in the plot? 

b) Okay, optimize the model (perhaps borrow the code from the 1D model above). Plot again. Better? 

c) How do the lines in the plots correspond to our distribution on functions?

d) Investigate the effect of the *number* of the inducing input points. Try the model with 4, 12, 20, 200 rows in Z.
Plot each: what happens?

e) Investigate the effect of the *locations* of the inducing points. Does it work if you initialize them far away from the model? What does the model do if you refuse to let it adapt Z (`m.Z.set_trainable(False)`)? 

bonus) Initialize the model with Z = np.zeros((10, 2)). What happens and why?

In [ ]:
# Your answers here!